In [12]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import RareLabelEncoder

In [3]:
data = pd.read_csv('creditApprovalUCIV2.csv')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
data.drop(labels=['A16'], axis=1), data['A16'],test_size=0.3,
random_state=0)

In [5]:
X_train.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,a,46.08,3.000,u,g,c,v,2.375,t,t,8,t,g,396.0,4159
303,a,15.92,2.875,u,g,q,v,0.085,f,f,0,f,g,120.0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,1,f,g,50.0,1187
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,0,f,g,100.0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,6,t,g,360.0,1332


In [6]:
X_train['A7'].value_counts() / len(X_train)

v          0.573499
h          0.209110
ff         0.084886
bb         0.080745
z          0.014493
j          0.010352
dd         0.010352
Missing    0.008282
n          0.006211
o          0.002070
Name: A7, dtype: float64

In [7]:
def find_frequent_labels(df, variable, tolerance):
    temp = df[variable].value_counts() / len(df)
    frequent = [x for x in temp.loc[temp>tolerance].index.values]
    return frequent

In [8]:
frequent_cat = find_frequent_labels(X_train, 'A7', 0.05)

In [9]:
print(frequent_cat)

['v', 'h', 'ff', 'bb']


In [10]:
X_train['A7'] = np.where(X_train['A7'].isin(frequent_cat), X_train['A7'], 'Rare')
X_test['A7'] = np.where(X_test['A7'].isin(frequent_cat), X_test['A7'], 'Rare')

In [11]:
X_train['A7'].value_counts() / len(X_train)

v       0.573499
h       0.209110
ff      0.084886
bb      0.080745
Rare    0.051760
Name: A7, dtype: float64

In [18]:
rare_encoder = RareLabelEncoder(tol=0.05, n_categories=4, variables=['A7'], replace_with='Rare')

In [19]:
rare_encoder.fit(X_train)

RareLabelEncoder(n_categories=4, variables=['A7'])

In [20]:
print(rare_encoder.encoder_dict_)

{'A7': Index(['v', 'h', 'ff', 'bb', 'Rare'], dtype='object')}


In [21]:
X_train_enc = rare_encoder.transform(X_train)
X_test_enc = rare_encoder.transform(X_test)